จาก [Notebook ที่แล้ว ที่เราทำ Image Classification ด้วย ResNet34](https://www.bualabs.com/archives/469/how-to-deep-learning-pets-cats-dogs-image-classification-with-fastai-v1-ep-1/) คราวนี้เราจะมาลองเทรนด้วย ResNet50 ซึ่งเป็นโมเดลที่ใหญ่ขึ้นซับซ้อนขึ้น ว่าความแม่นยำจะแตกต่างกันอย่างไร

# 0. Magic Commands


In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

# 1. Import Library

In [2]:
from fastai import *
from fastai.vision import *
from fastai.metrics import accuracy

# 2. ข้อมูล

ลดขนาด batch size เหลือ เนื่องจากโมเดลที่ขนาดใหญ่ขึ้น มีความต้องการ Memory ที่มากขึ้น ถ้าใช้ Batch Size เท่าเดิมจะทำให้ Error GPU Out of Memory ได้

In [4]:
path = untar_data(URLs.PETS)
path_images = path/'images'
filenames = get_image_files(path_images)

# 3. เตรียมข้อมูล

เพิ่มขนาดรูปจาก 224 เป็น 299

In [5]:
batchsize = 16
np.random.seed(0)
regex_pattern = r'/([^/]+)_\d+.jpg$'

databunch = ImageDataBunch.from_name_re(path_images, filenames, 
                                   regex_pattern, ds_tfms=get_transforms(), 
                                   size=299, bs=batchsize).normalize(imagenet_stats)

# 4. สร้างโมเดล

เปลี่ยนจาก models.resnet34 เป็น models.resnet50

In [ ]:
learner = cnn_learner(databunch, models.resnet50, metrics=accuracy)

# 5. เริ่มต้นเทรนโมเดล

In [ ]:
learner.fit_one_cycle(8)

เรามา Save Model ที่เราเพิ่งเทรนไปเก็บไว้ก่อน

In [ ]:
learner.save('resnet50-1')

โหลดโมเดลที่เรา Save ไว้ ขึ้นมาใหม่ เตรียมเทรนในขึ้นตอนถัดไป (ที่คอมเม้นท์ไว้ เพราะถ้าไม่ได้เปลี่ยนอะไรก็จะได้ไม่ต้องรัน)

In [ ]:
# learner.load('resnet50-1')

# 6. เทรนต่อ

In [ ]:
learner.lr_find()
learner.recorder.plot()

In [ ]:
learner.unfreeze()
learner.fit_one_cycle(3, max_lr=slice(1e-6,1e-4))

## สำเร็จแล้ว ##

เพียงแค่เวลา 1 นาทีเศษ เราเทรน Model ได้ accuracy ประมาณ 0.95 หรือ ความแม่นยำประมาณ 95.0%

เปรียบเทียบกับ ResNet34 ที่เราเทรนครั้งก่อน 94.0% ที่เทรนเฉพาะ Layer สุดท้าย 

# 7. ดูผลลัพธ์

In [ ]:
interpretation = ClassificationInterpretation.from_learner(learner)

In [ ]:
interpretation.most_confused(min_val=2)

# 8. สรุป


# Credit 

* [FastAI: Practical Deep Learning for Coders, v3 - Lesson 1](https://course.fast.ai/videos/?lesson=1)